In [ ]:
import importlib

import ipynb.fs.full.preprocessing0
import ipynb.fs.full.preprocessing05
#import ipynb.fs.full.prepossessing075_akistage
import preprocessing1
#import ipynb.fs.full.preprocessing2_BT
import preprocessing2_BT

import ipynb.fs.full.preprocessing25_BTcorr
import ipynb.fs.full.preprocessing3_smote
#import ipynb.fs.full.preprocessing4
import preprocessing4

#import ipynb.fs.full.runxgboost
import runxgboost

#import ipynb.fs.full.postprocessing1_SHAP

import postprocessing1_SHAP

import ipynb.fs.full.postprocessing3_collect

from ipynb.fs.full.slackbot import ping_slack
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from os.path import exists
import logging
import time

import utils_function
import seaborn as sns
from scipy import stats

import plot_utils
import pickle

import utils_code2name

In [ ]:
importlib.reload(ipynb.fs.full.preprocessing0)
importlib.reload(ipynb.fs.full.preprocessing05)
#importlib.reload(ipynb.fs.full.prepossessing075_akistage)
importlib.reload(preprocessing1)
importlib.reload(preprocessing2_BT)
importlib.reload(ipynb.fs.full.preprocessing25_BTcorr)
importlib.reload(ipynb.fs.full.preprocessing3_smote)
# #importlib.reload(ipynb.fs.full.preprocessing4)
importlib.reload(preprocessing4)
importlib.reload(runxgboost)
importlib.reload(postprocessing1_SHAP)
importlib.reload(ipynb.fs.full.postprocessing3_collect)
importlib.reload(postprocessing1_SHAP)
importlib.reload(utils_function)
importlib.reload(plot_utils)
importlib.reload(utils_code2name)

In [ ]:
site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
configs_variables = [utils_function.read_config(site) for site in site_list]

In [ ]:
plot_utils.geb_top_top(configs_variables)

In [ ]:
site, datafolder, home_directory = utils_function.get_commons(configs_variables[0])

In [ ]:
''' 
statistics
'''
onset_counts = list()
sumv = 0
sump = 0
#for site in site_list:
for site in site_list:
    onset = pd.read_parquet(datafolder+site+'/p0_onset_'+site+'.parquet')
    onset = onset.drop_duplicates()
    onset_count = onset[['ENCOUNTERID', 'AKI_STAGE']].groupby('AKI_STAGE').count()/onset.shape[0]
    sump += onset[['PATID']].drop_duplicates().shape[0]
    sumv += onset.shape[0]
    onset_count.columns = [site]
    onset_counts.append(onset_count)
onset_count = np.round(pd.concat(onset_counts,axis=1).T,2)


In [ ]:
'''
Collect cross_validationr results
'''
shap_datas = plot_utils.collect_cross_cv(configs_variables)

In [ ]:
#Avg model transportability
shap_datas_ext = shap_datas[shap_datas['site_m']!=  shap_datas['site_d']].groupby('site_m').mean().reset_index()
shap_datas_in = shap_datas[shap_datas['site_m']==  shap_datas['site_d']].groupby('site_m').mean().reset_index()

In [ ]:
# shap_datas_ext.rename({'roc':'roc_ext','prauc':'prauc_ext'}, inplace=True)
# shap_datas_ext.rename({'roc':'roc_in','prauc':'prauc_in'}, inplace=True)
shap_datasx = shap_datas_ext.merge(shap_datas_in,on='site_m',how='inner')
shap_datasx['trans'] = shap_datasx['roc_x']/shap_datasx['roc_y']
shap_datasx

In [ ]:
plot_utils.headmap(shap_datas, target='prauc', site_ano=True)

In [ ]:
plot_utils.headmap(shap_datas, site_ano=True)

In [ ]:
#plot_utils.create_radar_chart_with_confidence(shap_datas, site_ano=True)

In [ ]:
#plot_utils.create_radar_chart_with_confidence(shap_datas, target='prauc', site_ano=True)

In [ ]:
importlib.reload(plot_utils)

df_importances, df_importances_stat = plot_utils.get_importances_features_stat(configs_variables)

In [ ]:
code_translator = utils_code2name.code2name()
#code_translator.load_concept()
#omop_label = {x:code_translator.translate_omop(x) for x in df_importances_stat[df_importances_stat['Label_rank']!=-100].index}
#omop_label = {x:code_translator.custom_translate_omop_2022(x) for x in df_importances_stat[df_importances_stat['Label_rank']!=-100].index}

In [ ]:
df_importances_stat[['PX:' in x for x in df_importances_stat.index]].sort_values('Median')

In [ ]:
df_importances_stat.to_pickle('Fig2_plot_feature_importance_data.pkl')

In [ ]:
importlib.reload(plot_utils)
importlib.reload(utils_code2name)
code_translator = utils_code2name.code2name()
include_list=['PX:CH:J2543', 'PX:CH:36415']
modify_list=['LAB::LG7967-5', 'MED:ATC:B01AB', 'LAB::LG32857-1', 'LAB::LG50024-5', 'LAB::26505-8', 'LAB::18182-6', 'LAB::LG6139-2']
#modify_list = ['LAB::LG50024-5', 'LAB::LG1962-0']

plot_utils.plot_feature_importance(df_importances_stat, code_translator.custom_translate_omop_2022_2_no_lonic, modify_list, include_list)

In [ ]:
extracted_data = {
    'LAB::LG6657-3': 'Creatinine (LAB:LONIC:LG6657-3)',
    'LAB::48642-3': 'Glomerular filtration rate predicted among non-blacks (LAB:LONIC:48642-3)',}
extracted_data.get('LAB::LG6657-3', 'LAB::LG6657-3').rsplit(" ", 1)[0]

In [ ]:
#omop_label = {x:code_translator.translate_omop(x) for x in df['Feature Id no unit'].unique()}

df = df_importances.sort_values('rank', ascending=False).reset_index().groupby('site').head(10)
df['Feature Id'] = [code_translator.custom_translate_omop_2022_2(x) for x in df['Feature Id no unit']]
df = df[['Feature Id', 'site', 'rank']]
df = df.sort_values(by=['site', 'rank'], ascending=[True, False])
df['rank2'] = df.groupby('site').cumcount() + 1
df_pivot = df.pivot(index='site', columns='rank2', values='Feature Id')

df_pivot

In [ ]:
#df_importances[df_importances['Feature Id no unit']=='LAB::LG32850-6']

In [ ]:
importlib.reload(plot_utils)

df_importances, df_importances_stat = plot_utils.get_importances_features_stat(configs_variables)
external_heatmap_df = pd.read_csv('r2svg.csv').drop('Unnamed: 0',axis=1)

In [ ]:
df_importances

In [ ]:
importlib.reload(plot_utils)
include_list=['LAB::LG32892-8', 'AGE', 'LAB::LG4454-7']
modify_list=['LAB::LG6373-7', 'LAB::1962-0', 'MED:ATC:B01AB', 'LAB::LG32857-1', 'LAB::LG50024-5', 'LAB::26505-8', 'LAB::18182-6', 'LAB::LG6139-2']
plot_utils.plot_feature_importancer2(df_importances_stat, external_heatmap_df, code_translator.custom_translate_omop_2022_2_no_lonic, modify_list, include_list)